<image src="./images/dashboard.jpg" alt="A car dashboard with lots of new technical features." width="600">

You're working for a well-known car manufacturer who is looking at implementing LLMs into vehicles to provide guidance to drivers. You've been asked to experiment with integrating car manuals with an LLM to create a context-aware chatbot. They hope that this context-aware LLM can be hooked up to a text-to-speech software to read the model's response aloud.

As a proof of concept, you'll integrate several pages from a car manual that contains car warning messages and their meanings and recommended actions. This particular manual, stored as an HTML file, `mg-zs-warning-messages.html`, is from an MG ZS, a compact SUV. Armed with your newfound knowledge of LLMs and LangChain, you'll implement Retrieval Augmented Generation (RAG) to create the context-aware chatbot.

In [1]:
# Import the required packages
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import UnstructuredHTMLLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma

In [2]:
# Load the HTML as a LangChain document loader
loader = UnstructuredHTMLLoader(file_path="data/mg-zs-warning-messages.html")
car_docs = loader.load()

Split the Document

In [3]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)

chunks = splitter.split_documents(car_docs)
print(chunks[0].page_content)

Warning Message Procedure Cruise Control Fault Indicates that the cruise control system has detected a fault. Please consult an MG Authorised Repairer as soon as possible. Active Speed Limiter Fault Indicates that the active speed limit system has detected a fault. Contact an MG Authorised Repairer as soon as possible. Engine Coolant Temperature High High engine coolant temperature could result in severe damage. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately. Engine Coolant Temperature Sensor Fault Indicates that the engine coolant temperature sensor has failed. As soon as conditions permit, safely stop the vehicle and switch off the engine and contact an MG Authorised Repairer immediately.


Store the Embeddings

In [4]:
embedding_function = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

vectorstore = Chroma.from_documents(
    documents=car_docs,
    embedding=embedding_function
)

/var/folders/n5/mm68ph654jj0kwghx08j70l40000gp/T/ipykernel_11304/2457627876.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_function = HuggingFaceEmbeddings(
/Users/crypticsy/anaconda3/envs/genai/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


import error: No module named 'triton'


Create a retriever

In [5]:
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}  # Number of documents to retrieve
)

Initialize the LLM and Prompt template

In [ ]:
from langchain_community.llms import Ollama

# Using Ollama applications within the operating system
llm = Ollama(model="llama3")

prompt = ChatPromptTemplate.from_template(""" You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
                                              If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
                                              \nQuestion: {question} 
                                              \nContext: {context} 
                                              \nAnswer:""")


/var/folders/n5/mm68ph654jj0kwghx08j70l40000gp/T/ipykernel_11304/722147542.py:3: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3")


Defining the RAG Chain

In [14]:
from langchain.schema import StrOutputParser

rag_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
    }
    | prompt
    | llm
    | StrOutputParser()
)

Invoke RAG chain

In [15]:
user_query = "The Gasoline Particular Filter Full warning has appeared. What does this mean and what should I do about it?"

result = rag_chain.invoke(user_query)
print(result)

The Gasoline Particular Filter Full warning means that your gasoline particulate filter is full and needs to be replaced or cleaned. You should consult an MG Authorised Repairer as soon as possible to resolve the issue.
